## Mounter

In [11]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Extractor

In [0]:
import pandas as pd
import numpy as np
import json
import csv
import bz2
import os
import sys

In [0]:
def extractor(fle,writer,up_l,down_l,IP_det):
  total=[]
  source_file=bz2.BZ2File(fle,'r')
  prev='ip'
  temp_l=[]
  temp_l_2=[]
  rtt=[]
  for i,line in enumerate(source_file):
    if i<down_l: 
      continue
    if i==up_l: 
      break
    #print(i)
    data=json.loads(line)
    if 'from' not in data.keys() or 'dst_addr' not in data.keys():
      continue
    src_addr=data['from']
    dest_addr=data["dst_addr"]
    if IP_det.get(dest_addr) is None:
      continue
    print("A")
    temp=data['result']
    for iter in temp:
      if "result" not in iter.keys():
        continue
      data_temp=iter['result']
      for ext in data_temp:
        if "from" in ext.keys():
          if prev==str(ext["from"]): continue
          temp_l.append(ext["from"])
          temp_l_2.append(ext["rtt"])
          prev=str(ext["from"])
    if len(temp_l)==0:
      continue
    writer.write(str(i)+","+str(src_addr)+","+str(dest_addr)+"\n")
    total.append(temp_l) 
    rtt.append(temp_l_2) 
    temp_l=[]
    temp_l_2=[]
  down_l=up_l
  up_l=up_l+20000
  if i<down_l-1:
    flag=1
  else:
    flag=0

  return total,down_l,up_l,flag,rtt  

In [0]:
def merge(name_w,total,rtt):
  df=pd.read_csv(name_w)
  l=len(df)
  df['Hops']=total
  df['Latency']=rtt
  df.to_csv(name_w,index=False)

In [0]:
def caller_ext():
  Path="drive/My Drive/iitkgp_summer_project"
  fldr="traceroute"
  print(Path+'/'+fldr)
  files=os.listdir(Path+"/"+fldr)
  up_l=20000
  down_l=0
  z=0
  fle_1="temp.bz2"
  fle=Path+"/"+fle_1
  print(fle)
  j=0
  flag=0
  file_l='netflix.json'
  print(Path+'/'+file_l)
  f=open(Path+'/'+file_l,"r")
  IP_det=json.loads(f.read())
  print(len(IP_det))
  name=Path+'/'+'netflix'
  print(name)
  if os.path.exists(name):
   
      print("error")
      exit(0)
  else:
      os.mkdir(name)

  counter_n=1
  while(flag==0):
    

    
      name_w=name+'/IP_netflix'+"_"+str(counter_n)+".csv"
      print(name_w)
      text=open(name_w,"w")

      text.write("index,source,dest\n")
      text.close()
      writer=open(name_w,'a')


      total,down_l,up_l,flag,rtt=extractor(fle,writer,up_l,down_l,IP_det)
      j+=1
      writer.close()
      merge(name_w, total,rtt)  
      counter_n+=1

In [0]:
caller_ext()

drive/My Drive/iitkgp_summer_project/traceroute
drive/My Drive/iitkgp_summer_project/temp.bz2
drive/My Drive/iitkgp_summer_project/netflix.json
88064
drive/My Drive/iitkgp_summer_project/netflix
error
drive/My Drive/iitkgp_summer_project/netflix/IP_netflix_1.csv
drive/My Drive/iitkgp_summer_project/netflix/IP_netflix_2.csv
drive/My Drive/iitkgp_summer_project/netflix/IP_netflix_3.csv
drive/My Drive/iitkgp_summer_project/netflix/IP_netflix_4.csv
drive/My Drive/iitkgp_summer_project/netflix/IP_netflix_5.csv
drive/My Drive/iitkgp_summer_project/netflix/IP_netflix_6.csv
drive/My Drive/iitkgp_summer_project/netflix/IP_netflix_7.csv
drive/My Drive/iitkgp_summer_project/netflix/IP_netflix_8.csv
drive/My Drive/iitkgp_summer_project/netflix/IP_netflix_9.csv
drive/My Drive/iitkgp_summer_project/netflix/IP_netflix_10.csv
A
drive/My Drive/iitkgp_summer_project/netflix/IP_netflix_11.csv
drive/My Drive/iitkgp_summer_project/netflix/IP_netflix_12.csv
drive/My Drive/iitkgp_summer_project/netflix/IP_ne

KeyboardInterrupt: ignored

In [0]:
import pandas as pd
import os
import ipaddress
import json
Path="drive/My Drive/iitkgp_summer_project"

file_1=Path+'/'+'netflix_v6.csv'
file_2=Path+'/'+'netflix_v6.json'

df=pd.read_csv(file_1)
i=0
all_ips=[]
l=[]
while i<len(df):
	l.append(df.iloc[i]['ip'])
	i+=1
print(l)
ip_list={}
for i in  l:
		#k+=1
		#if k==len(i)-3:
		#	continue
		temp_ip=[str(ip) for ip in ipaddress.IPv6Network(i)]
		all_ips.append(temp_ip)
print(len(all_ips))
for i in all_ips:
		for j in i:
			if ip_list.get(j) is None:
				ip_list[j]=0

with open(file_2,'w') as fp:
	json.dump(ip_list, fp)